In [1]:
from matplotlib import pyplot as plt
import cv2


In [2]:
import math
def fixPixelValues(px):
    # convert the RGB values into floating point to avoid an overflow that will give me wrong answers
    return [ float(px[0]), float(px[1]), float(px[2]) ]


# This is a useful function that given a list of (x,y) values,
# draw a series of red lines between each coordinate and next to
# show the path in the image
def drawPath(img, path, pThick=2):
    v = path[0]
    x0, y0 = v[0], v[1]
    for v in path:
        x, y = v[0], v[1]
        cv2.line(img,(x,y), (x0,y0), (255,0,0),pThick)
        x0, y0 = x,y

In [3]:
class Vertex: # This is the outline for a vertex data structure

    def __init__ (self,  i, j):
        self.x = i # The x coordinate
        self.y = j  # The y coordinate
        self.d = float('inf') # the shortest path estimate
        self.processed = False # Has this vertex's final shortest path distance been computed
        # this is important for Dijksatra's algorithm
        # We will track where the vertex is in the priority queue.
        self.idx_in_priority_queue = -1 # The index of this vertex in the queue
        self.pi = None # the parent vertex in the shortest path tree.
        print("## Vertex (i, J):  (", Vertex, i, j,")")

    def reset(self):
        self.d = float('inf')
        self.processed = False # Has this vertex's final shortest path distance been computed
        # this is important for Dijksatra's algorithm
        # We will track where the vertex is in the priority queue.
        self.idx_in_priority_queue = -1 # The index of this vertex in the queue
        self.pi = None # the parent vertex in the shortest path tree.
        print("## reset: idx_in_priority_queue, pi ##", idx_in_priority_queue, pi)

In [4]:
# However, if you want Dijkstra efficiently, we will need a priority queue
# We will provide you with a heap data structure from course 1.
class PriorityQueue:
    # Constructor:  Implement a empty heap data structure
    def __init__(self):
        self.q = [None] # pad it with one element
        print("## PriorityQueue > __init__ ##  self.q: ", self.q)

    # Function: insert
    # Insert a vertex v of type Vertex into the queue.
    # Remember to set the field `idx_in_priority_queue` and
    # keep updating it.
    def insert(self, v):
        n = len(self.q)
        print(" ## PriorityQueue > insert ## ---------------")
        print(" n (before append 'v'): ", n, "v: ", v)

        self.q.append(v)
        v.idx_in_priority_queue = n
        print(" n (after append 'v'): ", len(self.q))

        self.bubble_up(n)
        print("bubble_up(n)", n)
        #self.check_invariant()

    # Function: swap two elements in the priority queue.
    # Remember to swap the vertices at positions i and j
    # But also remember to update the positions of the vertices in the
    # priority queue.
    # You can use this to implement bubble_up and bubble_down
    def swap(self, i, j):
        print("swap two elements in the priority queue")
        print("i, j: ", i, j)

        tmp = self.q[i]
        print("tmp = self.q[i]: ", tmp)
        self.q[i] = self.q[j]
        self.q[i].idx_in_priority_queue = i
        self.q[j] = tmp
        print("self.q[j]: ", self.q[j])
        self.q[j].idx_in_priority_queue = j
        print("after swap", "i, j: ", i, j)


    # Function: bubble_up
    # bubble up an element j
    # until min heap property is restored.
    def bubble_up(self, j):
        assert j >= 1
        assert j < len(self.q)
        print("## Function: bubble_up ##  ")
        print("bubble_up(before) - j, len(self.q): ", j, len(self.q))
        if j == 1:
            return
        val = self.q[j].d
        parent_idx = j // 2
        parent_val = self.q[parent_idx].d
        print("val, parent_idx, parent_val: ", val, parent_idx, parent_val)

        if val < parent_val:
            print("if val < parent_val then swap j: ", j, "then bubble up parent_idx:", parent_idx)
            self.swap(j, parent_idx)
            print("self.swap(j, parent_idx): ", j, parent_idx)
            self.bubble_up(parent_idx)
            print("self.bubble_up(parent_idx): ", parent_idx)

        return

    # Function: bubble_down
    # Bubble down an element j until
    # min heap property is restored.
    def bubble_down(self, j):
        n = len(self.q)
        left_child_idx = 2 * j
        right_child_idx = 2 * j + 1

        print("## PriorityQueue > bubble_down (before) ##")
        print("n: ", n, "left_child_idx: ", left_child_idx, "right_child_idx: ", right_child_idx)

        if left_child_idx >= n:
            return
        if right_child_idx >= n:
            child_idx = left_child_idx
            child_d = self.q[left_child_idx].d
        else:
            (child_d, child_idx) = min ( (self.q[left_child_idx].d, left_child_idx),
                                         (self.q[right_child_idx].d, right_child_idx)
                                       )
        if self.q[j].d > child_d:
            self.swap(j, child_idx)
            self.bubble_down(child_idx)

        print("## PriorityQueue > bubble_down (after) ##")
        print("n: ", n, "left_child_idx: ", left_child_idx, "right_child_idx: ", right_child_idx)

        return

    # Function: get_and_delete_min
    # Find the minimum weight vertex and delete it from the heap.
    # return the deleted vertex back
    def get_and_delete_min(self):
        n = len(self.q)

        print(" ## PriorityQueue > get_and_delete_min ## ")
        print("n = len(self.q): ", n)

        assert n > 1
        v = self.q[1]

        print("v = self.q[1]: ", v)

        if n > 2:
            self.q[1] = self.q[n-1]
            self.q[n-1].idx_in_priority_queue = 1
            del self.q[n-1]
            self.bubble_down(1)
        #self.check_invariant()
        return v

    # Is the heap empty?
    def is_empty(self):
       print(" ## def is_empty, len(self.q) == 1 ? ")
       return len(self.q) == 1

    # This is a useful function since in Dijkstra
    # the weight of a vertex updates on the fly.
    # We will need to call this to update the vertex weight.
    def update_vertex_weight(self, v):
        j = v.idx_in_priority_queue
        n = len(self.q)
        print("## PriorityQueue > update_vertex_weight(v): ##")
        print("j = v.idx_in_priority_queue: ", j, "n = len(self.q): ", n)
        assert j >= 0 and j < n
        self.bubble_down(j)
        self.bubble_up(j)
        # self.check_invariant()

In [5]:
class DirectedGraphFromImage:
    def __init__(self, img):
        self.img = img
        self.coords2vertex = {} # construct a dictionary that maps coordinates [(i,j)] to corresponding vertices in graph


    def get_vertex_from_coords(self, i, j):
        if (i,j) in self.coords2vertex: # is pixel (i,j) already there?
            print("## DirectedGraphFromImage > get_vertex_from_coords ##")
            print("(i,j): ", i, j)
            print("[(self.coords2vertex)]: ", self.coords2vertex)
            return self.coords2vertex[(i,j)] # if yes, just return the vertex corresponding
        v = Vertex(i, j)
        self.coords2vertex[(i,j)] = v
        print("self.coords2vertex[(i,j)] = v: ", v)
        return v

    ## Given (x,y) coordinates of two neighboring pixels, calculate the edge weight.
    # We take the squared euclidean distance between the pixel values and add 0.1
    def getEdgeWeight(self, u, v):
        img = self.img
        # get edge weight for edge between u, v
        i0,j0 = u.x, u.y
        i1,j1 = v.x, v.y
        height, width, _ = img.shape
        # First make sure that the edge is legit
        # Edges can only go from each pixel to neighboring pixel
        assert i0 >= 0 and j0 >= 0 and i0 < width and j0 < height # pixel position valid?
        assert i1 >= 0 and j1 >= 0 and i1 < width and j1 < height # pixel position valid?
        assert -1 <= i0 - i1 <= 1 # edge between node and neighbor?
        assert -1 <= j0 - j1 <= 1
        px1 = fixPixelValues(img[j0,i0])
        px2 = fixPixelValues(img[j1,i1])

        print("## DirectedGraphFromImage > getEdgeWeight ##")
        print("px1[0], px1[1], px2[0], px2[1]:" , px1[0], px1[1], px2[0], px2[1])

        return 0.1 + (px1[0] - px2[0])**2 + (px1[1] - px2[1])**2 + (px1[2]- px2[2])**2

    # Function: get_list_of_neighbors
    # Given a vertex in the graph, get its list of neighbors
    #  I.e, for given vertex `vert` return a list [(v1, w1), (v2, w2),..,(vk,wk)]
    #  Such that vert has an edge to v1 with weight w1, edge to v2 with weight w2 and ...
    #   edge to vk with weight wk
    # Note that rather than build an adjacency list up front, we simply call this function
    # to get the neighbors of a vertex.
    def get_list_of_neighbors(self, vert):
        img = self.img
        i = vert.x
        j = vert.y
        height, width, _ = img.shape
        lst = []
        if i > 0:
             # Get the adjacent vertex directly to the WEST
            # What is the weight of the edge from pixel (i,j) to (i-1,j)
            v0 = self.get_vertex_from_coords(i-1, j)
            w0 = self.getEdgeWeight(vert, v0)
            # Append the adjacent vertex and its weight.
            lst.append((v0, w0))
        if j > 0:
            # Get the adjacent vertex directly to the SOUTH
            v1 = self.get_vertex_from_coords(i, j-1)
            w1 = self.getEdgeWeight(vert, v1)
            # Append the adjacent vertex and its weight.
            lst.append((v1, w1))
        if i < width-1:
            # EAST
            v2 = self.get_vertex_from_coords(i+1, j)
            w2 = self.getEdgeWeight( vert, v2)
            lst.append((v2, w2))
        if j < height-1:
            # NORTH
            v3 = self.get_vertex_from_coords(i, j+1)
            w3 = self.getEdgeWeight(vert, v3)
            lst.append((v3, w3))
        print("## DirectedGraphFromImage > get_list_of_neighbors ## ")
        print("vert (i, j): ", vert.x, vert.y)
        print("height, width, _ = img.shape: ", height, width, _)
        print("lst: ", lst)
        print("v0, w0: ", v0, w0)
        print("v1, w1: ", v1, w1)
        print("v2, w2: ", v2, w2)
        print("v3, w3: ", v3, w3)
        return lst

In [6]:
# Function: computeShortestPath
# Let us implement Dijkstra's algorithm
# graph - instance of the DirectedGraphFromImage class
# source - a vertex that is the source (i,j) pixel coordinates
# dest - a vertex that is the destination (i,j) pixel coordinates
def computeShortestPath(graph, source_coordinates, dest_coordinates):
    # your code here
    frontier = PriorityQueue()
    print("## computeShortestPath > frontier = PriorityQueue() ##")
    print("frontier: ", frontier)
    i0, j0 = source_coordinates
    source = graph.get_vertex_from_coords(i0,j0)
    print(" {source} = ", source)

    i5,j5 = dest_coordinates
    destination = graph.get_vertex_from_coords(i5,j5)
    print(" {destination} = ",destination)

    #insert source into Priority Queue
    source.d = 0
    frontier.insert(source)
    print("## computeShortestPath > frontier.insert(source) ##")
    print("frontier: ", frontier)
    frontier.update_vertex_weight(source)
    print("## frontier.update_vertex_weight(source) ##")

    while not frontier.is_empty():
        print("## computeShortestPath > while not frontier.is_empty(): ##")

        pop = frontier.get_and_delete_min()
        print("pop = frontier.get_and_delete_min(): ", pop)

        pop.processed = True
        print("pop.processed = True")


        if [pop.x, pop.y] == [destination.x, destination.y]:
            totaldist = pop.d
            print("totaldist = pop.d: ", totaldist)
            destiny = pop
            print("destiny = pop: ", destiny)
            break

        else:
            frontier4 = graph.get_list_of_neighbors(pop)
            print("frontier4 = graph.get_list_of_neighbors(pop): ", frontier4)

            for i in range(len(frontier4)):
                print("for [i: ", i,"], len(frontier4): ", len(frontier4))
                ve, wt = frontier4[i]
                print("ve = ", ve, "wt: ", wt)

                if  ve.processed == False:
                    if ve.d > pop.d + wt:
                        ve.d = pop.d + wt
                        ve.pi = pop
                        print("ve.d (",ve.d,") > then = pop.d(",pop.d,") + wt(",wt,")")
                        print("ve.pi: ", ve.pi, "ve: ", ve)
                        try:
                            frontier.insert(ve)
                            print("try frontier.insert(ve):")
                            print("{ve}, {frontier}: ", ve, frontier)
                        except:
                            print("An exception occurred")
                        frontier.update_vertex_weight(ve)
                        print("## frontier.update_vertex_weight(ve)## ve = ", ve)

                        frontier.bubble_up(ve.idx_in_priority_queue)
                        print("## frontier.bubble_up(ve.idx_in_priority_queue) ##", ve.idx_in_priority_queue)
                        print(["ve.x, ve.y, ve.d, : ", ve.x, ve.y, ve.d])

    ve = destiny
    print("ve = destiny: ", ve)
    pathback = [(ve.x, ve.y)]
    print("pathback = [(ve.x, ve.y)]: ", pathback)

    while  ve != source:
        print("(while  ve != source:) ve = ve.pi: ", ve.pi)
        ve = ve.pi
        print("pathback.append((ve.x,ve.y)): ", ve.x, ve.y)
        pathback.append((ve.x,ve.y))
        print("pathback: ", pathback)

    pathback.reverse()
    print("pathback.reverse(): ", pathback)
    print("totaldist: ", totaldist)
    return (pathback , totaldist)

In [7]:
class DummyGraphClass:
    def __init__(self, adj_list, verts):
        self.verts=verts
        self.adj_list = adj_list


    def get_vertex_from_coords(self, i, j):
        assert (i,j) in self.verts
        print("## DummyGraphClass > get_vertex_from_coords ##")
        print("(i,j): ", i, j, " in {self.verts}: ", self.verts)
        return self.verts[(i,j)]

    def get_list_of_neighbors(self, vert):
        coords = (vert.x, vert.y)
        if coords in self.adj_list:
            print("## DummyGraphClass > get_list_of_neighbors ## ")
            print("(vert.x, vert.y):  ", vert.x, vert.y)
            print("self.adj_list[(vert.x, vert.y)]:  ", self.adj_list[(vert.x, vert.y)])
            return self.adj_list[(vert.x, vert.y)]

        else:
            return []

In [8]:
# Test 1
verts = {(i,j): Vertex(i,j) for i in range(3) for j in range(3)}
adj_list= {}
print("verts = ", verts)

## Vertex (i, J):  ( <class '__main__.Vertex'> 0 0 )
## Vertex (i, J):  ( <class '__main__.Vertex'> 0 1 )
## Vertex (i, J):  ( <class '__main__.Vertex'> 0 2 )
## Vertex (i, J):  ( <class '__main__.Vertex'> 1 0 )
## Vertex (i, J):  ( <class '__main__.Vertex'> 1 1 )
## Vertex (i, J):  ( <class '__main__.Vertex'> 1 2 )
## Vertex (i, J):  ( <class '__main__.Vertex'> 2 0 )
## Vertex (i, J):  ( <class '__main__.Vertex'> 2 1 )
## Vertex (i, J):  ( <class '__main__.Vertex'> 2 2 )
verts =  {(0, 0): <__main__.Vertex object at 0x79a2938b1190>, (0, 1): <__main__.Vertex object at 0x79a29387cfd0>, (0, 2): <__main__.Vertex object at 0x79a293811c50>, (1, 0): <__main__.Vertex object at 0x79a293a1a210>, (1, 1): <__main__.Vertex object at 0x79a29220acd0>, (1, 2): <__main__.Vertex object at 0x79a2929a8190>, (2, 0): <__main__.Vertex object at 0x79a292208090>, (2, 1): <__main__.Vertex object at 0x79a292209050>, (2, 2): <__main__.Vertex object at 0x79a29220bcd0>}


In [9]:
def connect_nodes(src, dest, weight):
    v1 = src
    v2 = verts[dest]

    if v1 in adj_list:
        adj_list[v1].append((v2, weight))

    else:
        adj_list[v1] = [(v2, weight)]






In [10]:
# Let's build a graph
connect_nodes((0,0),(0,1),1.0)
connect_nodes((0,0),(1,0),0.5)
connect_nodes((1,0),(0,1), 0.5)
connect_nodes((0,1),(0,0), 0.5)
connect_nodes((1,0),(1,1), 0.5)
connect_nodes((1,1), (2,2), 0.25)
connect_nodes((1,1),(1,2), 0.5)
connect_nodes((1,1),(2,1), 1.2)
connect_nodes((2,1), (2,2), 0.25)
connect_nodes((1,2), (2,2), 0.25)

print("adj_list: ", adj_list)

adj_list:  {(0, 0): [(<__main__.Vertex object at 0x79a29387cfd0>, 1.0), (<__main__.Vertex object at 0x79a293a1a210>, 0.5)], (1, 0): [(<__main__.Vertex object at 0x79a29387cfd0>, 0.5), (<__main__.Vertex object at 0x79a29220acd0>, 0.5)], (0, 1): [(<__main__.Vertex object at 0x79a2938b1190>, 0.5)], (1, 1): [(<__main__.Vertex object at 0x79a29220bcd0>, 0.25), (<__main__.Vertex object at 0x79a2929a8190>, 0.5), (<__main__.Vertex object at 0x79a292209050>, 1.2)], (2, 1): [(<__main__.Vertex object at 0x79a29220bcd0>, 0.25)], (1, 2): [(<__main__.Vertex object at 0x79a29220bcd0>, 0.25)]}


In [11]:
graph = DummyGraphClass(adj_list, verts)

In [12]:
path, dist = computeShortestPath(graph, (0,0), (2,2))

## PriorityQueue > __init__ ##  self.q:  [None]
## computeShortestPath > frontier = PriorityQueue() ##
frontier:  <__main__.PriorityQueue object at 0x79a292217710>
## DummyGraphClass > get_vertex_from_coords ##
(i,j):  0 0  in {self.verts}:  {(0, 0): <__main__.Vertex object at 0x79a2938b1190>, (0, 1): <__main__.Vertex object at 0x79a29387cfd0>, (0, 2): <__main__.Vertex object at 0x79a293811c50>, (1, 0): <__main__.Vertex object at 0x79a293a1a210>, (1, 1): <__main__.Vertex object at 0x79a29220acd0>, (1, 2): <__main__.Vertex object at 0x79a2929a8190>, (2, 0): <__main__.Vertex object at 0x79a292208090>, (2, 1): <__main__.Vertex object at 0x79a292209050>, (2, 2): <__main__.Vertex object at 0x79a29220bcd0>}
 {source} =  <__main__.Vertex object at 0x79a2938b1190>
## DummyGraphClass > get_vertex_from_coords ##
(i,j):  2 2  in {self.verts}:  {(0, 0): <__main__.Vertex object at 0x79a2938b1190>, (0, 1): <__main__.Vertex object at 0x79a29387cfd0>, (0, 2): <__main__.Vertex object at 0x79a293811c50>

In [13]:
print(path)

[(0, 0), (1, 0), (1, 1), (2, 2)]
